# 应该用datetime模块来处理本地时间，而不是用time模块

In [1]:
import logging

## time模块

内置的time模块中，有个名叫localtime函数，它可以把UNIX时间戳转换为与宿主计算机的时区相符的当地时间。

In [2]:
from time import localtime, strftime

now = 1407694710
local_tuple = localtime(now)
time_format = '%Y-%m-%d %H:%M:%S'
time_str = strftime(time_format, local_tuple)
print(time_str)

2014-08-11 02:18:30


程序还需要做反向处理，把用户输入的本地时间，转换为UTC时间。

In [3]:
from time import mktime, strptime

time_tuple = strptime(time_str, time_format)
utc_now = mktime(time_tuple)
print(utc_now)

1407694710.0


把某个地区的当地时间转换为另一个时区的当地时间

In [4]:
parse_format = '%Y-%m-%d %H:%M:%S %Z'
depart_sfo = '2014-05-01 15:45:16 GMT'
time_tuple = strptime(depart_sfo, parse_format)
time_str = strftime(time_format, time_tuple)
print(time_str)

2014-05-01 15:45:16


In [5]:
try:
    arrival_nyc = '2014-05-01 23:33:24 EDT'
    time_tuple = strptime(arrival_nyc, time_format)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-5-2a32494428f3>", line 3, in <module>
    time_tuple = strptime(arrival_nyc, time_format)
  File "E:\DevSoft\Python\Python37\lib\_strptime.py", line 571, in _strptime_time
    tt = _strptime(data_string, format)[0]
  File "E:\DevSoft\Python\Python37\lib\_strptime.py", line 362, in _strptime
    data_string[found.end():])
ValueError: unconverted data remains:  EDT


## datetime模块

In [6]:
from datetime import datetime, timezone

#把UTC格式的当前时间，转换为计算机所用的本地时间
now = datetime(2014, 8, 10, 18, 18, 30)
now_utc = now.replace(tzinfo=timezone.utc)
now_local = now_utc.astimezone()
print(now_local)

2014-08-11 02:18:30+08:00


In [7]:
# 把本地时间转换成UTC格式的UNIX时间戳
time_str = '2014-08-10 11:18:30'
now = datetime.strptime(time_str, time_format)
time_tuple = now.timetuple()
utc_now = mktime(time_tuple)
print(utc_now)

1407640710.0


为了有效使用pytz模块，应该先把当地时间转换为UTC，然后针对UTC值进行datetime操作，最后再把UTC转回当地时间

In [8]:
# 把航班到达纽约的时间，转换为UTC格式的datetime对象
import pytz
arrival_nyc = '2014-05-01 23:33:24'
nyc_dt_naive = datetime.strptime(arrival_nyc, time_format)
eastern = pytz.timezone('US/Eastern')
nyc_dt = eastern.localize(nyc_dt_naive)
utc_dt = pytz.utc.normalize(nyc_dt.astimezone(pytz.utc))
print(utc_dt)

2014-05-02 03:33:24+00:00


得到UTC格式的datetime之后，再把它转换成旧金山当地时间

In [9]:
pacific = pytz.timezone('US/Pacific')
sf_dt = pacific.normalize(utc_dt.astimezone(pacific))
print(sf_dt)

2014-05-01 20:33:24-07:00


还可以把这个时间，轻松转换成尼泊尔当地时间。

In [10]:
nepal = pytz.timezone('Asia/Katmandu')
nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))
print(nepal_dt)

2014-05-02 09:18:24+05:45
